## Geodepy Tutorial: Time-Dependent Datum Transformations

This tutorial demonstrates the use of GeodePy for transformations between time-dependent dynamic datums. We'll work through each of the steps required to tranform a coordinate in ITRF2005 at epoch 2000.0 to ITRF2014 at 2020.0 on the Australian Plate, then build a function containing these steps to simplify this process.

Start by importing the following Functions and Modules (datetime is used for the representation of epochs):

In [ ]:
from datetime import date
from geodepy.constants import itrf2014_to_gda2020, itrf2014_to_itrf2005
from geodepy.transform import *

We'll define a point for transformation. For this exercise, we'll use a point called Bob which is in terms of ITRF2005 at epoch 2000.0 and is shown using a UTM projection. The parts below are (zone, easting, northing, ellipsoidal height):

In [ ]:
bob = (53, 386353.2371, 7381852.2967, 587.5814)
bob

#### Part 1: Getting Things in the Right Format
To convert between time-dependent datums, we need our coordinates to be in Cartesian (XYZ) format. To get Bob in this format, we start by converting the UTM part (zone, easting and northing) into Geographic (latitude and longitude) format:

In [ ]:
lat, lon, psf, grid_conv = grid2geo(bob[0], bob[1], bob[2])
lat, lon

We can now use Bob's Latitude, Longitude and Ellipsoidal Height to calculate Cartesian Coordinates for Bob. These are still in terms of ITRF2005 at epoch 2000.0:

In [ ]:
x, y, z = llh2xyz(lat, lon, bob[3])
x, y, z

#### Part 2: Moving to the Reference Epoch
Transformations between datums in GeodePy use Transformation Objects which are contained in the Constants module. These contain parameters used in Conformal (Helmert) 7 and 15 parameter transformations. To go between ITRF2005 and ITRF2014, we use `itrf2014_to_itrf2005`, but because we're going the other way we use `-itrf2014_to_itrf2005`:

In [ ]:
-itrf2014_to_itrf2005

Before we can use this, we need to get our coordinates into the Reference Epoch of the transformation we plan to use (in this case, 2010.0 or the 1st Jan 2010). Because Bob is in Australia, we need to use the Australian Plate Motion Model to do this. It's parameters are in `itrf2014_to_gda2020`:

In [ ]:
itrf2014_to_gda2020

It also has a reference epoch; for `itrf2014_to_gda2020`, it's 2020.0. So to move coordinates from 2000.0 to 2010.0 (+10.0 years) using `itrf2014_to_gda2020` we need to use `conform14` with a `to_epoch` 10.0 years past our reference epoch (2020.0 + 10.0 = 2030.0):

In [ ]:
x, y, z

In [ ]:
x2, y2, z2, _ = conform14(x, y, z, date(2010, 1, 1), itrf2014_to_gda2020)
x2, y2, z2

We now have Bob's Cartesian Coordinates in ITRF2005 at epoch 2010.0!

#### Part 3: The Transformation
Now that we have Bob's coordinates in the reference epoch of `itrf2014_to_itrf2005`, we can perform this transformation. Because we're not changing epochs in this part, we can use the `conform7` function which ignores the time-dependent parameters of our transformation:

In [ ]:
x3, y3, z3, _ = conform7(x2, y2, z2, -itrf2014_to_itrf2005)
x3, y3, z3

This gives us Bob's Cartesian Coordinates in ITRF2014 at epoch 2010.0.
#### Part 4: Moving to the Final Epoch
The final tranformation is moving Bob's epoch to it's final destination (2020.0) using `itrf2014_to_gda2020`. As the period of movement (2010.0 to 2020.0) is the same (+10.0 years), we use the same `to_epoch` of 2030.0 as in Part 2:

In [ ]:
x4, y4, z4, _ = conform14(x3, y3, z3, date(2010, 1, 1), itrf2014_to_gda2020)
x4, y4, z4

These coordinates are now in terms of ITRF2014 at epoch 2020.0
#### Part 5: Getting Things in their Original Format
To bring everything back into the format we started, we'll step through the process in Part 1 in reverse. So we'll start by converting our Cartesian Coordinates to Geographic format:

In [ ]:
lat_end, lon_end, ell_ht_end = xyz2llh(x4, y4, z4)
lat_end, lon_end, ell_ht_end

Next, we'll convert the latitude and longitude into UTM grid coordinates:

In [ ]:
hem_end, zone_end, east_end, north_end, psf_end, grid_conv_end = geo2grid(lat_end, lon_end)
zone_end, east_end, north_end

Combining this with our ellipsoidal height from above, we get the final coordinate:

In [ ]:
zone_end, east_end, north_end, ell_ht_end

We can then compare the starting coordinate to the final coordinate and see the difference:

In [ ]:
bob_end = (zone_end, east_end, north_end, ell_ht_end)
bob_end

In [ ]:
bob

#### Part 6: Simplifying steps into a single function
To wrap up this tutorial, we'll combine all of the steps above into a single function:

In [ ]:
def bobtransform(zone, east, north, ell_ht):
    lat, lon, psf, grid_conv = grid2geo(zone, east, north)
    x, y, z = llh2xyz(lat, lon, ell_ht)
    x2, y2, z2, _ = conform14(x, y, z, date(2030, 1, 1), itrf2014_to_gda2020)
    x3, y3, z3, _ = conform7(x2, y2, z2, -itrf2014_to_itrf2005)
    x4, y4, z4, _ = conform14(x3, y3, z3, date(2030, 1, 1), itrf2014_to_gda2020)
    lat_end, lon_end, ell_ht_end = xyz2llh(x4, y4, z4)
    hem_end, zone_end, east_end, north_end, psf_end, grid_conv_end = geo2grid(lat_end, lon_end)
    return zone_end, east_end, north_end, ell_ht_end

newbob = bobtransform(bob[0], bob[1], bob[2], bob[3])
newbob